<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/Hamiltonian_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In quantum computing, "running" the Schrödinger Equation is known as **Hamiltonian Simulation**.

The Schrödinger Equation tells us that if we want to know the state of a system at time $t$ ($|\psi(t)\rangle$), we must apply the time-evolution operator to the starting state:

$$ |\psi(t)\rangle = e^{-i\hat{H}t} |\psi(0)\rangle $$

### The Algorithm: Trotterization
Since quantum computers run on digital gates, we cannot usually apply the big matrix $e^{-i\hat{H}t}$ all at once. Instead, we break time $t$ into small steps (slices) and simulate the Hamiltonian term by term. This is called **Trotterization**.

If your Hamiltonian (Energy) is $H = A + B$ (e.g., kinetic + potential energy), the algorithm approximates the evolution as:
$$ e^{-i(A+B)t} \approx \left( e^{-iA \frac{t}{n}} e^{-iB \frac{t}{n}} \right)^n $$

### The Quiskit Code
Here is a simulation of a 2-qubit system evolving under the **Ising Hamiltonian** ($H = Z_0 Z_1 + X_0 + X_1$). This effectively "solves" the Schrödinger equation for this system over time.




In [1]:
!pip install qiskit qiskit_aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def simulate_schrodinger_evolution(time_t, trotter_steps):
    """
    Simulates the Schrödinger Equation: |ψ(t)> = e^(-iHt)|ψ(0)>
    Hamiltonian H = Z0*Z1 (Interaction) + X0 (Transverse Field) + X1
    """

    # 1. Initialize Circuit
    qc = QuantumCircuit(2)

    # PREPARE INITIAL STATE |ψ(0)>
    # Let's start in |10> (Qubit 0=1, Qubit 1=0)
    qc.x(0)

    # 2. Define Time Step (dt)
    dt = time_t / trotter_steps

    # 3. Apply Evolution (Trotterization Loop)
    # We repeat the small time slice 'n' times
    for _ in range(trotter_steps):

        # --- Part A: Evolution under Transverse Field (X terms) ---
        # Operator: e^{-i X dt} corresponds to Rotation Rx(2*dt)
        qc.rx(2 * dt, 0)
        qc.rx(2 * dt, 1)

        # --- Part B: Evolution under Interaction (ZZ term) ---
        # Operator: e^{-i Z0Z1 dt} corresponds to Rzz(2*dt)
        # We build Rzz using CNOTs and Rz
        qc.cx(0, 1)
        qc.rz(2 * dt, 1)
        qc.cx(0, 1)

        # Barrier to visualize steps clearly in drawings
        qc.barrier()

    # 4. Measure
    qc.measure_all()

    return qc

# --- Parameters ---
target_time = 1.5   # How long to evolve the system
steps = 5           # Granularity of the simulation (more steps = more accurate)

# Generate Circuit
circuit = simulate_schrodinger_evolution(target_time, steps)

# Run Simulation
simulator = AerSimulator()
compiled_circuit = transpile(circuit, simulator)
job = simulator.run(compiled_circuit, shots=2000)
counts = job.result().get_counts()

# --- Output ---
print(f"--- Schrödinger Evolution Simulation ---")
print(f"Hamiltonian: H = Z0Z1 + X0 + X1")
print(f"Time t: {target_time}, Steps: {steps}")
print(f"Initial State: |10>")
print(f"Final Counts: {counts}")

# Plotting (Optional if running locally)
# plot_histogram(counts)
print("\n--- Algorithm Structure (First Step) ---")
print(circuit.draw(output='text', idle_wires=False, fold=60))


--- Schrödinger Evolution Simulation ---
Hamiltonian: H = Z0Z1 + X0 + X1
Time t: 1.5, Steps: 5
Initial State: |10>
Final Counts: {'11': 17, '01': 846, '00': 16, '10': 1121}

--- Algorithm Structure (First Step) ---
           ┌───┐   ┌─────────┐                      ░ »
   q_0: ───┤ X ├───┤ Rx(0.6) ├──■───────────────■───░─»
        ┌──┴───┴──┐└─────────┘┌─┴─┐┌─────────┐┌─┴─┐ ░ »
   q_1: ┤ Rx(0.6) ├───────────┤ X ├┤ Rz(0.6) ├┤ X ├─░─»
        └─────────┘           └───┘└─────────┘└───┘ ░ »
meas: 2/══════════════════════════════════════════════»
                                                      »
«        ┌─────────┐                      ░ ┌─────────┐»
«   q_0: ┤ Rx(0.6) ├──■───────────────■───░─┤ Rx(0.6) ├»
«        ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐ ░ ├─────────┤»
«   q_1: ┤ Rx(0.6) ├┤ X ├┤ Rz(0.6) ├┤ X ├─░─┤ Rx(0.6) ├»
«        └─────────┘└───┘└─────────┘└───┘ ░ └─────────┘»
«meas: 2/══════════════════════════════════════════════»
«                                                  

### Understanding the Translation

1.  **$\hat{H}$ (The Hamiltonian):**
    In the code, $\hat{H}$ is represented by the gates inside the loop.
    *   Term $X$ becomes `rx(angle)`.
    *   Term $ZZ$ becomes `cx` $\to$ `rz` $\to$ `cx`.

2.  **$t$ (Time):**
    Time is translated into the **Rotation Angle** of the gates.
    *   Angle $\theta = 2 \cdot H_{\text{coeff}} \cdot dt$.
    *   The longer the time $t$, the more the qubits rotate.

3.  **$\partial/\partial t$ (Dynamics):**
    The `for` loop represents the continuous flow of time. By breaking it into discrete steps (`dt`), we digitally integrate the differential equation.

### Is this useful?
Yes. This is widely considered the **"Killer App"** for quantum computers. Classical computers struggle to solve the Schrödinger equation for large molecules (chemistry) because the equation gets exponentially complex. A quantum computer maps the molecule's Hamiltonian directly onto the qubits and just lets it evolve naturally.